In [32]:
from src.utils import create_spark_session
spark = create_spark_session()

In [35]:
spark.sql("SHOW BRANCHES legal.documents_snapshot").show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'legal': missing 'FUNCTIONS'.(line 1, pos 14)

== SQL ==
SHOW BRANCHES legal.documents_snapshot
--------------^^^


In [36]:
spark.sql("select * from legal.documents_snapshot.branch_staging limit 10").show()

+-----------+-------------+-------------------+--------------------+--------+---------+------+--------------+--------------------+--------------------+--------------------+------------------+------------+
|document_id|document_type|       generated_at|              source|language|file_size|method|schema_version|  metadata_file_path|            raw_text|           file_path|          batch_id|      job_id|
+-----------+-------------+-------------------+--------------------+--------+---------+------+--------------+--------------------+--------------------+--------------------+------------------+------------+
|    doc_000|        dummy|2024-01-01 10:00:00|soli_legal_docume...|      en|     1024| spark|           1.0|s3a://raw/docs/le...|This is a sample ...|s3a://raw/docs/le...|batch_20240115_001|test_job_001|
+-----------+-------------+-------------------+--------------------+--------+---------+------+--------------+--------------------+--------------------+--------------------+--------

In [37]:
spark.sql("select * from legal.documents_snapshot limit 10").show()


+--------------------+-------------+--------------------+--------------------+--------+---------+----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|         document_id|document_type|        generated_at|              source|language|file_size|    method|schema_version|  metadata_file_path|            raw_text|           file_path|            batch_id|              job_id|
+--------------------+-------------+--------------------+--------------------+--------+---------+----------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|04d7a600-a2db-453...|     contract|2025-07-07 16:12:...|soli_legal_docume...|      en|     1756|sequential|            v1|s3a://raw/docs/le...|LEGAL DOCUMENT 00...|s3a://raw/docs/le...|batch_legal_inser...|legal_insert_1751...|
|77130b22-0fb5-482...|     contract|2025-07-07 16:12:...|soli_legal_docume...|      

In [38]:
spark.sql("select * from legal.documents_snapshot.refs").show()

+-------+------+-------------------+-----------------------+---------------------+----------------------+
|   name|  type|        snapshot_id|max_reference_age_in_ms|min_snapshots_to_keep|max_snapshot_age_in_ms|
+-------+------+-------------------+-----------------------+---------------------+----------------------+
|staging|BRANCH|2253963032283617332|                   NULL|                 NULL|                  NULL|
|   main|BRANCH|6836768928478080649|                   NULL|                 NULL|                  NULL|
+-------+------+-------------------+-----------------------+---------------------+----------------------+



In [39]:
spark.sql("DESCRIBE TABLE legal.documents_snapshot").show()

+------------------+---------+--------------------+
|          col_name|data_type|             comment|
+------------------+---------+--------------------+
|       document_id|   string|Unique identifier...|
|     document_type|   string|Type of legal doc...|
|      generated_at|timestamp|When the document...|
|            source|   string|Source system ide...|
|          language|   string|Language of the d...|
|         file_size|   bigint|Size of the file ...|
|            method|   string|Processing method...|
|    schema_version|   string|Version of the me...|
|metadata_file_path|   string|Path to the metad...|
|          raw_text|   string|Raw text content ...|
|         file_path|   string|Path to the docum...|
|          batch_id|   string|Links to batch_jo...|
|            job_id|   string|Job identifier fo...|
+------------------+---------+--------------------+



In [29]:
df_branch = spark.read \
    .format("iceberg") \
    .option("branch", "staging") \
    .load("legal.documents_snapshot")

df_branch.show(truncate=False)

+-----------+-------------+-------------------+-----------------------------+--------+---------+------+--------------+------------------------------------------------------------+----------------------------------------------------------+----------------------------------------------------------+------------------+------------+
|document_id|document_type|generated_at       |source                       |language|file_size|method|schema_version|metadata_file_path                                          |raw_text                                                  |file_path                                                 |batch_id          |job_id      |
+-----------+-------------+-------------------+-----------------------------+--------+---------+------+--------------+------------------------------------------------------------+----------------------------------------------------------+----------------------------------------------------------+------------------+------------+
|doc_000  

In [ ]:
spark.sql("select count(*) from legal.documents_snapshot IN BRANCH staging limit 10").show()

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'BRANCH'.(line 1, pos 42)

== SQL ==
select * from legal.documents_snapshot IN BRANCH staging limit 10
------------------------------------------^^^


In [3]:
spark.sql("SELECT distinct(document_type) FROM legal.documents").show()

+---------------+
|  document_type|
+---------------+
|  legal_opinion|
|     legal_memo|
|       contract|
|   court_filing|
|policy_document|
+---------------+



In [15]:
spark.sql("SELECT job_id, batch_id, count(*) FROM legal.documents GROUP BY job_id, batch_id").show()

+--------------------+----------------+--------+
|              job_id|        batch_id|count(1)|
+--------------------+----------------+--------+
|legal_insert_1751...|batch_1751839771|     100|
+--------------------+----------------+--------+



In [ ]:
result = spark.sql("SELECT raw_text FROM legal.documents WHERE document_type = 'contract' LIMIT 1")
result.show(truncate=False)

SyntaxError: invalid syntax (1435888384.py, line 1)